# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [59]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [64]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)
print(a)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]


In [61]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [62]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [48]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [49]:
np.linalg.matrix_rank(a1)

6

In [77]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


C:\Users\sofya\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\sofya\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
C:\Users\sofya\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
C:\Users\sofya\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [78]:
def if_nonzero(A):
    #флаг опущен
    flag = 0
    n = A.shape[0] + 1
    for i in range(1,n):
        if np.linalg.det(A[:i,:i]) == 0: #если минор нулевой, поднимаем флаг
            flag += 1
            break
    if flag:
        return False
    return True



### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [90]:
#lu-разложение с выбором ведущего элемента:
#изменения в алгоритме следующие:
#при переходе к новой строке в ней выбирается ведущий(наибольший)
#элемент, и столбцы меняются таким образом, чтобы ведущий элемент находился в рабочем столбце
#матрицу перестановок преобразуем параллельно из единичной

def diy_lup(a):
    N = a.shape[0]
    U = a.copy()
    L = np.eye(N)
    P = np.eye(N)
    
    for j in range(N-1):
        lam = np.eye(N)
        P_temp = np.eye(N)
        tmp = np.argmax(abs(U[j,j:])) + j #столбец, в котором находится максимальный элемент
        P_temp[j, j] = 0
        P_temp[tmp,tmp] = 0
        P_temp[j, tmp] = 1
        P_temp[tmp, j] = 1
        U = U @ P_temp.T
        
        gamma = U[j + 1:, j] / U[j, j]
        lam[j + 1:, j] = -gamma
        U = lam @ U
        
        lam[j + 1:, j] = gamma
        L = L @ lam
        
        P = P @ P_temp.T
    return L, U, P
    


In [93]:
def initial_matrix(L, U, P):
    return L @ U @ P.T

In [91]:
L, U, P = diy_lup(a1)
print("Тест для a", if_nonzero(a))
print("Тест для a1", if_nonzero(a1))
print("Нижнетреугольная матрица L = ","\n", L)
print("Верхнетреугольная матрица U = ","\n", U)
print("Матрица перестановок P =", "\n", P)
print("Исходная матрица через разложение:","\n", "a1 = ", "\n", initial_matrix(L, U, P))

Тест для a True
Тест для a1 False
Нижнетреугольная матрица L =  
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.143 1.    0.    0.    0.   ]
 [1.    1.2   1.179 1.    0.    0.   ]
 [1.    1.231 1.294 1.605 1.    0.   ]
 [1.    1.25  1.375 2.007 2.076 1.   ]]
Верхнетреугольная матрица U =  
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -2.475e-01 -1.437e-01 -6.259e-02]
 [ 0.000e+00  2.220e-16  2.220e-16 -9.247e-02 -4.755e-02 -1.920e-02]
 [ 0.000e+00 -1.344e-16 -3.565e-16  0.000e+00  1.841e-03  1.181e-03]
 [ 0.000e+00 -1.667e-16  2.941e-16 -1.567e-17 -4.337e-19 -1.233e-05]]
Матрица перестановок P = 
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]]
Исходная матрица через разложение: 
 a1 =  
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    3.   

In [92]:
L2, U2, P2 = diy_lup(a)
print("Нижнетреугольная матрица L = ","\n", L2)
print("Верхнетреугольная матрица U = ","\n", U2)
print("Матрица перестановок P =", "\n", P2)
print("Исходная матрица через разложение:", "\n", "a = ","\n", initial_matrix(L2, U2, P2))


Нижнетреугольная матрица L =  
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.143 1.    0.    0.    0.   ]
 [1.    1.2   1.65  1.    0.    0.   ]
 [1.    1.231 2.09  2.01  1.    0.   ]
 [1.    1.25  2.406 2.875 2.346 1.   ]]
Верхнетреугольная матрица U =  
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -2.475e-01 -1.437e-01 -6.259e-02]
 [ 0.000e+00  2.220e-16  0.000e+00  2.421e-02  2.021e-02  1.030e-02]
 [ 0.000e+00 -2.242e-16  0.000e+00  0.000e+00 -6.462e-04 -5.096e-04]
 [ 0.000e+00 -1.124e-16  0.000e+00 -1.238e-18  0.000e+00  6.730e-06]]
Матрица перестановок P = 
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]]
Исходная матрица через разложение: 
 a =  
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1